# Rewards and Returns

Welcome back! Now that we understand MDPs, let's dive into the heart of reinforcement learning: **rewards and returns**. This is how we measure success!

## What You'll Learn

By the end of this notebook, you'll understand:
- The difference between rewards and returns (with a paycheck analogy!)
- Why discounting matters (the "bird in hand" principle)
- How to calculate returns step by step
- How to design good reward functions
- Common pitfalls like reward hacking

**Prerequisites:** Notebooks 1-2

**Time:** ~25 minutes

---
## The Big Picture: Paycheck vs Savings

Think about your financial life:

```
    ┌────────────────────────────────────────────────────────┐
    │            REWARD vs RETURN: A Money Analogy           │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  REWARD (r_t):                                         │
    │    Your PAYCHECK this month                            │
    │    → Immediate feedback for this time step             │
    │    → Example: $5,000 this month                        │
    │                                                         │
    │  RETURN (G_t):                                         │
    │    Your TOTAL LIFETIME SAVINGS                         │
    │    → Sum of all future paychecks                       │
    │    → What you really care about!                       │
    │                                                         │
    │  KEY INSIGHT:                                          │
    │    You don't optimize for ONE paycheck.                │
    │    You optimize for your TOTAL WEALTH over time!       │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

This is exactly what RL agents do:
- They receive **rewards** (immediate feedback)
- But they optimize for **returns** (long-term success)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch

# Visualize the difference between rewards and returns
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Individual Rewards (Paychecks)
ax1 = axes[0]
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
paychecks = [5000, 5000, 5500, 5000, 6000, 5500]  # Monthly rewards

bars = ax1.bar(months, paychecks, color='#4CAF50', edgecolor='black', linewidth=2)
ax1.set_ylabel('Amount ($)', fontsize=12)
ax1.set_title('REWARDS (Individual Paychecks)\n"How much did I earn THIS month?"', 
              fontsize=14, fontweight='bold')
ax1.set_ylim(0, 7000)

for bar, pay in zip(bars, paychecks):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100, 
             f'${pay:,}', ha='center', fontsize=10)

# Right: Cumulative Return (Savings)
ax2 = axes[1]
savings = np.cumsum(paychecks)  # Running total

ax2.fill_between(range(len(months)), savings, color='#2196F3', alpha=0.3)
ax2.plot(range(len(months)), savings, 'b-o', linewidth=3, markersize=10)
ax2.set_xticks(range(len(months)))
ax2.set_xticklabels(months)
ax2.set_ylabel('Total Amount ($)', fontsize=12)
ax2.set_title('RETURN (Cumulative Savings)\n"How much have I earned IN TOTAL?"', 
              fontsize=14, fontweight='bold')

for i, sav in enumerate(savings):
    ax2.annotate(f'${sav:,}', (i, sav), textcoords="offset points", 
                 xytext=(0, 10), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("="*60)
print("KEY DIFFERENCE")
print("="*60)
print(f"\nReward (r): What you get at ONE time step")
print(f"            Examples: {paychecks}")
print(f"\nReturn (G): TOTAL of all rewards from now on")
print(f"            After June: ${sum(paychecks):,}")
print(f"\nRL agents try to MAXIMIZE RETURN, not just individual rewards!")
print("="*60)

---
## The Chess Sacrifice: Why Returns > Rewards

Here's a powerful example of why long-term thinking matters:

```
    ┌────────────────────────────────────────────────────────┐
    │                   CHESS SACRIFICE                       │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  Situation: You can sacrifice your Queen to set up     │
    │             a guaranteed checkmate in 3 moves.          │
    │                                                         │
    │  Option A (greedy - maximize immediate reward):        │
    │    Keep your Queen (worth 9 points)                    │
    │    Immediate reward: 0 (no loss)                       │
    │    Long-term: Opponent might win                       │
    │                                                         │
    │  Option B (strategic - maximize return):               │
    │    Sacrifice Queen (lose 9 points now)                 │
    │    Immediate reward: -9                                │
    │    BUT: Guaranteed checkmate = WIN = +1000!            │
    │                                                         │
    │  Return for Option A: 0 + uncertain future             │
    │  Return for Option B: -9 + 1000 = +991                 │
    │                                                         │
    │  THE SACRIFICE IS BETTER because the RETURN is higher! │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

**This is why RL agents optimize for RETURNS, not immediate REWARDS!**

In [ ]:
# Demonstrate the chess sacrifice scenario

print("CHESS EXAMPLE: Should you sacrifice your Queen?")
print("="*55)

# Option A: Keep Queen (greedy)
print("\n🅰️  OPTION A: Keep your Queen (greedy approach)")
rewards_A = [0, 0, 0, 0, 0]  # No immediate loss, uncertain outcome
print(f"   Rewards over time: {rewards_A}")
print(f"   Return (sum): {sum(rewards_A)}")
print(f"   Outcome: Uncertain - might win, might lose")

# Option B: Sacrifice Queen for checkmate
print("\n🅱️  OPTION B: Sacrifice Queen for checkmate")
rewards_B = [-9, 0, 0, 1000]  # Lose queen (-9), then win (+1000)
print(f"   Rewards over time: {rewards_B}")
print(f"   Return (sum): {sum(rewards_B)}")
print(f"   Outcome: GUARANTEED WIN!")

print("\n" + "="*55)
print("LESSON: A negative immediate reward can lead to a")
print("        much better TOTAL RETURN!")
print("="*55)

---
## Calculating Returns: Step by Step

The **return** at time t is the sum of all future rewards:

```
G_t = r_t + r_{t+1} + r_{t+2} + ... + r_T
```

Let's calculate this with a simple example:

In [ ]:
def calculate_simple_return(rewards):
    """
    Calculate the total return (undiscounted).
    This is just the sum of all rewards.
    """
    return sum(rewards)

def calculate_returns_at_each_step(rewards):
    """
    Calculate the return starting from each time step.
    G_t = r_t + r_{t+1} + ... + r_T
    """
    T = len(rewards)
    returns = []
    
    for t in range(T):
        # Sum from time t to the end
        G_t = sum(rewards[t:])
        returns.append(G_t)
    
    return returns

# Example: A game episode
# Small negative rewards for each step (-1), big reward for winning (+100)
rewards = [-1, -1, -1, -1, -1, 100]

print("EXAMPLE: A Simple Game Episode")
print("="*55)
print(f"\nRewards at each step: {rewards}")
print(f"  • Steps 0-4: -1 each (cost of taking an action)")
print(f"  • Step 5: +100 (won the game!)")

returns = calculate_returns_at_each_step(rewards)

print("\n" + "-"*55)
print("Return at each time step:")
print("-"*55)

for t in range(len(rewards)):
    future_rewards = rewards[t:]
    print(f"  G_{t} = {' + '.join(map(str, future_rewards))} = {returns[t]}")

print("\n" + "="*55)
print("Notice: Earlier time steps have HIGHER returns because")
print("        they include MORE future rewards!")
print("="*55)

In [ ]:
# Visualize returns at each time step
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Individual rewards
ax1 = axes[0]
time_steps = list(range(len(rewards)))
colors = ['#f44336' if r < 0 else '#4CAF50' for r in rewards]

bars = ax1.bar(time_steps, rewards, color=colors, edgecolor='black', linewidth=2)
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax1.set_xlabel('Time Step (t)', fontsize=12)
ax1.set_ylabel('Reward (r_t)', fontsize=12)
ax1.set_title('Individual Rewards at Each Step', fontsize=14, fontweight='bold')

for i, (bar, r) in enumerate(zip(bars, rewards)):
    y_pos = bar.get_height() + (5 if r > 0 else -8)
    ax1.text(bar.get_x() + bar.get_width()/2, y_pos, str(r), ha='center', fontsize=11)

# Right: Returns from each time step
ax2 = axes[1]
ax2.bar(time_steps, returns, color='#2196F3', edgecolor='black', linewidth=2)
ax2.set_xlabel('Time Step (t)', fontsize=12)
ax2.set_ylabel('Return (G_t)', fontsize=12)
ax2.set_title('Return (Total Future Reward) from Each Step', fontsize=14, fontweight='bold')

for i, G in enumerate(returns):
    ax2.text(i, G + 2, f'G_{i}={G}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("The return DECREASES as we move forward in time because")
print("there are fewer future rewards left to collect!")

---
## Discounting: A Bird in the Hand

Here's an important question:

> **Would you rather have $100 today or $100 one year from now?**

Most people prefer $100 today! Why?

```
    ┌────────────────────────────────────────────────────────┐
    │            WHY DISCOUNT FUTURE REWARDS?                 │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  1. UNCERTAINTY                                        │
    │     The future is uncertain. You might not get the     │
    │     reward if something goes wrong!                    │
    │                                                         │
    │  2. OPPORTUNITY COST                                   │
    │     Money today can be invested. $100 now could        │
    │     become $110 next year!                             │
    │                                                         │
    │  3. MATHEMATICAL NECESSITY                             │
    │     For infinite-horizon problems, we need discounting │
    │     to keep the return finite.                         │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

We model this preference with a **discount factor γ (gamma)**:

```
G_t = r_t + γ·r_{t+1} + γ²·r_{t+2} + γ³·r_{t+3} + ...
```

Where:
- **γ = 1**: No discounting (all rewards equally valuable)
- **γ = 0**: Only care about immediate reward
- **γ = 0.9**: Typical value (future matters, but less than present)

In [ ]:
def calculate_discounted_return(rewards, gamma=0.99):
    """
    Calculate the DISCOUNTED return.
    
    G_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...
    
    Args:
        rewards: List of rewards [r_0, r_1, r_2, ...]
        gamma: Discount factor (0 to 1)
    
    Returns:
        The discounted return G_0
    """
    G = 0
    for t, r in enumerate(rewards):
        G += (gamma ** t) * r
    return G

# Example: Same rewards, different discount factors
rewards = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]  # $10 each step

print("DISCOUNTING EXAMPLE")
print("="*55)
print(f"\nRewards: {rewards}")
print(f"(Earning $10 for 10 time steps)")
print("\n" + "-"*55)
print(f"{'Gamma':<10} {'Return':<15} {'Interpretation'}")
print("-"*55)

gamma_values = [1.0, 0.99, 0.9, 0.5, 0.0]
interpretations = [
    "No discounting (all equally valued)",
    "Very patient (cares about future)",
    "Moderate discounting",
    "Impatient (future worth half)",
    "Extremely myopic (only NOW matters)"
]

for gamma, interp in zip(gamma_values, interpretations):
    G = calculate_discounted_return(rewards, gamma)
    print(f"{gamma:<10.2f} ${G:<14.2f} {interp}")

print("\n" + "="*55)
print("Notice: With gamma=0, only the FIRST reward counts!")
print("        With gamma=1, ALL rewards are counted equally.")
print("="*55)

In [ ]:
# Visualize how gamma affects the "weight" of future rewards

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Weight decay over time for different gammas
ax1 = axes[0]
time_steps = np.arange(20)
gammas = [0.99, 0.9, 0.7, 0.5]
colors = ['#1e88e5', '#43a047', '#fb8c00', '#e53935']

for gamma, color in zip(gammas, colors):
    weights = gamma ** time_steps
    ax1.plot(time_steps, weights, label=f'γ = {gamma}', linewidth=3, color=color)
    ax1.scatter(time_steps, weights, s=30, color=color)

ax1.set_xlabel('Time Steps into Future', fontsize=12)
ax1.set_ylabel('Weight (γ^t)', fontsize=12)
ax1.set_title('How Much Do Future Rewards Count?', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.1)
ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax1.text(10, 0.52, 'Half weight line', fontsize=9, color='gray')

# Right: Cumulative weight (effective horizon)
ax2 = axes[1]

for gamma, color in zip(gammas, colors):
    cumulative = [sum(gamma ** np.arange(t+1)) for t in time_steps]
    ax2.plot(time_steps, cumulative, label=f'γ = {gamma}', linewidth=3, color=color)

# Add theoretical limits
ax2.axhline(y=1/(1-0.9), color='#43a047', linestyle=':', alpha=0.5)
ax2.axhline(y=1/(1-0.5), color='#e53935', linestyle=':', alpha=0.5)

ax2.set_xlabel('Time Steps into Future', fontsize=12)
ax2.set_ylabel('Cumulative Weight', fontsize=12)
ax2.set_title('"Effective Horizon": How Far Ahead Do We Look?', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("EFFECTIVE HORIZON")
print("="*60)
print("\nThe formula 1/(1-γ) gives the 'effective horizon':")
print("  • γ = 0.5:  1/(1-0.5) = 2 steps ahead")
print("  • γ = 0.9:  1/(1-0.9) = 10 steps ahead")
print("  • γ = 0.99: 1/(1-0.99) = 100 steps ahead")
print("\nHigher γ = Looking further into the future!")
print("="*60)

---
## The Recursive Trick: A Powerful Insight

Here's a beautiful mathematical property of returns:

```
G_t = r_t + γ·G_{t+1}
```

In plain English:
> **"My total future reward = Immediate reward + Discounted future reward"**

This is like saying:
> "My total wealth = Today's paycheck + Discounted value of all future paychecks"

This recursive formula is **incredibly important** - it's the foundation of many RL algorithms!

In [ ]:
def calculate_returns_recursive(rewards, gamma=0.99):
    """
    Calculate returns using the RECURSIVE formula:
    G_t = r_t + gamma * G_{t+1}
    
    We work BACKWARDS from the end!
    """
    T = len(rewards)
    returns = [0] * T
    
    # Start from the LAST time step
    returns[T-1] = rewards[T-1]  # G_T = r_T (no future rewards)
    
    # Work backwards through time
    for t in range(T-2, -1, -1):  # T-2, T-3, ..., 1, 0
        returns[t] = rewards[t] + gamma * returns[t+1]
    
    return returns

# Example: The big reward is at the END
rewards = [0, 0, 0, 0, 100]  # Zero rewards, then BIG reward at the end
gamma = 0.9

print("RECURSIVE RETURN CALCULATION")
print("="*60)
print(f"\nRewards: {rewards}")
print(f"Gamma (γ): {gamma}")
print(f"\nFormula: G_t = r_t + γ × G_{t+1}")
print("\n" + "-"*60)
print("Working BACKWARDS from the end:")
print("-"*60)

returns = calculate_returns_recursive(rewards, gamma)

# Show the calculation step by step
T = len(rewards)
for t in range(T-1, -1, -1):
    if t == T-1:
        print(f"  t={t}: G_{t} = r_{t} = {rewards[t]:.2f}")
        print(f"         (Last step: no future rewards)")
    else:
        print(f"  t={t}: G_{t} = r_{t} + γ×G_{t+1}")
        print(f"         = {rewards[t]} + {gamma}×{returns[t+1]:.2f}")
        print(f"         = {returns[t]:.2f}")

print("\n" + "-"*60)
print(f"Final returns: {[f'{r:.2f}' for r in returns]}")
print("\n" + "="*60)
print("NOTICE: The final reward 'propagates backward' through time!")
print("        Even at t=0, we get some value from the distant reward.")
print("="*60)

In [ ]:
# Visualize how value propagates backward

fig, ax = plt.subplots(figsize=(12, 6))

# Draw time steps
positions = range(len(rewards))
ax.bar(positions, returns, color='#2196F3', edgecolor='black', linewidth=2, alpha=0.7)

# Add reward annotations
for i, (r, G) in enumerate(zip(rewards, returns)):
    ax.text(i, G + 3, f'G={G:.1f}', ha='center', fontsize=11, fontweight='bold')
    ax.text(i, -5, f'r={r}', ha='center', fontsize=10, color='red')

# Draw arrows showing backward propagation
for i in range(len(rewards)-1, 0, -1):
    ax.annotate('', xy=(i-0.8, returns[i-1]*0.7), 
                xytext=(i-0.2, returns[i]*0.7),
                arrowprops=dict(arrowstyle='->', color='orange', lw=2))

ax.set_xlabel('Time Step (t)', fontsize=12)
ax.set_ylabel('Return (G_t)', fontsize=12)
ax.set_title(f'Backward Propagation of Value (γ = {gamma})\n'
             f'The reward at t=4 "flows backward" to earlier states', 
             fontsize=14, fontweight='bold')
ax.set_ylim(-10, max(returns) + 15)

# Add legend
ax.text(2, max(returns) + 10, '← Orange arrows show value flowing backward', 
        fontsize=10, color='orange')

plt.tight_layout()
plt.show()

---
## Reward Function Design: The Art of Shaping Behavior

Designing reward functions is one of the most important (and tricky!) parts of RL.

### Sparse vs Dense Rewards

```
    ┌─────────────────────────────────────────────────────────────┐
    │                  SPARSE vs DENSE REWARDS                    │
    ├─────────────────────────────────────────────────────────────┤
    │                                                             │
    │  SPARSE REWARDS:                                           │
    │    Only get reward at the goal                             │
    │    rewards = [0, 0, 0, 0, 0, +100]                         │
    │    ✓ Simple and clear                                      │
    │    ✗ Hard to learn (no guidance along the way)             │
    │                                                             │
    │  DENSE REWARDS:                                            │
    │    Get reward at every step                                │
    │    rewards = [-1, -1, -1, -1, -1, +100]                    │
    │    ✓ Easier to learn (constant feedback)                   │
    │    ✗ Might cause unintended behavior!                      │
    │                                                             │
    └─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Compare sparse vs dense rewards for the same task

# Task: Navigate from (0,0) to goal at (4,4)
# Path taken: (0,0) -> (1,0) -> (2,0) -> (3,0) -> (4,0) -> (4,1) -> ... -> (4,4)

def manhattan_distance(pos, goal):
    """Manhattan distance between two points."""
    return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])

def sparse_reward(pos, goal):
    """Only reward at the goal."""
    return 100 if pos == goal else 0

def dense_reward_step(pos, goal):
    """Negative reward for each step (encourages speed)."""
    if pos == goal:
        return 100
    return -1  # Cost for each step

def dense_reward_distance(pos, goal):
    """Reward based on how close we are to goal."""
    return -manhattan_distance(pos, goal)

def dense_reward_progress(prev_pos, pos, goal):
    """Reward for making progress toward goal."""
    if pos == goal:
        return 100
    prev_dist = manhattan_distance(prev_pos, goal)
    curr_dist = manhattan_distance(pos, goal)
    return prev_dist - curr_dist  # +1 if closer, -1 if farther

# Example path
goal = (4, 4)
path = [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]

print("REWARD FUNCTION COMPARISON")
print("="*70)
print(f"\nPath: {' -> '.join([str(p) for p in path])}")
print(f"Goal: {goal}")
print("\n" + "-"*70)

sparse_rewards = [sparse_reward(p, goal) for p in path]
dense_step_rewards = [dense_reward_step(p, goal) for p in path]
dense_dist_rewards = [dense_reward_distance(p, goal) for p in path]
dense_prog_rewards = [dense_reward_progress(path[max(0,i-1)], p, goal) for i, p in enumerate(path)]

print(f"{'Reward Type':<20} {'Rewards':<45} {'Total Return'}")
print("-"*70)
print(f"{'Sparse':<20} {str(sparse_rewards):<45} {sum(sparse_rewards)}")
print(f"{'Dense (step cost)':<20} {str(dense_step_rewards):<45} {sum(dense_step_rewards)}")
print(f"{'Dense (distance)':<20} {str(dense_dist_rewards):<45} {sum(dense_dist_rewards)}")
print(f"{'Dense (progress)':<20} {str(dense_prog_rewards):<45} {sum(dense_prog_rewards)}")

print("\n" + "="*70)
print("OBSERVATIONS:")
print("  • Sparse: Agent only knows it succeeded AT THE END")
print("  • Dense (step): Agent learns to reach goal FAST")
print("  • Dense (distance): Agent learns to stay CLOSE to goal")
print("  • Dense (progress): Agent learns to MOVE TOWARD goal")
print("="*70)

In [ ]:
# Visualize the different reward structures

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

reward_types = [
    ('Sparse Reward', sparse_rewards, '#9c27b0'),
    ('Dense: Step Cost', dense_step_rewards, '#f44336'),
    ('Dense: Distance', dense_dist_rewards, '#ff9800'),
    ('Dense: Progress', dense_prog_rewards, '#4caf50')
]

for ax, (title, rewards, color) in zip(axes.flat, reward_types):
    bars = ax.bar(range(len(rewards)), rewards, color=color, edgecolor='black', linewidth=2, alpha=0.7)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_xlabel('Step', fontsize=11)
    ax.set_ylabel('Reward', fontsize=11)
    ax.set_title(f'{title}\nTotal Return = {sum(rewards)}', fontsize=12, fontweight='bold')
    
    # Add value labels
    for i, (bar, r) in enumerate(zip(bars, rewards)):
        if r != 0:
            y_pos = bar.get_height() + (3 if r > 0 else -5)
            ax.text(bar.get_x() + bar.get_width()/2, y_pos, str(r), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

---
## Reward Hacking: When Good Intentions Go Wrong

**Reward hacking** is when an agent finds unexpected ways to maximize reward that don't align with what you actually wanted.

```
    ┌────────────────────────────────────────────────────────┐
    │                   REWARD HACKING EXAMPLES               │
    ├────────────────────────────────────────────────────────┤
    │                                                         │
    │  ROBOT WALKING:                                        │
    │    Intended: Reward for forward speed                  │
    │    Hack: Robot falls forward very fast!                │
    │                                                         │
    │  GAME PLAYING:                                         │
    │    Intended: Reward for high score                     │
    │    Hack: Found a glitch to get infinite points!        │
    │                                                         │
    │  CLEANING ROBOT:                                       │
    │    Intended: Reward for collecting trash               │
    │    Hack: Creates mess to have more trash to collect!   │
    │                                                         │
    │  CHATBOT (RLHF):                                       │
    │    Intended: Reward for helpful responses              │
    │    Hack: Overly agreeable, tells you what you want!    │
    │                                                         │
    └────────────────────────────────────────────────────────┘
```

**Lesson:** Agents optimize EXACTLY what you reward - which may not be what you MEANT!

In [ ]:
# Demonstration: Reward Hacking in a Simple Game

print("REWARD HACKING EXAMPLE: The Boat Racing Game")
print("="*60)

print("""
Scenario: A boat racing game where the AI should race around a track.

DESIGNER'S INTENT:
  Race around the track as fast as possible.

REWARD FUNCTION (naive):
  +1 for each checkpoint passed
  
WHAT HAPPENED:
  The AI discovered it could drive in circles, repeatedly
  hitting the same few checkpoints, without ever completing
  a lap - and score MORE points than actually racing!
""")

# Simulated rewards
print("-"*60)
print("Comparison of Strategies:")
print("-"*60)

# Intended behavior: Complete laps
intended_rewards = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 10 checkpoints = 1 lap
print(f"INTENDED (race around track): {intended_rewards}")
print(f"  Time: 10 steps, Return: {sum(intended_rewards)}")

# Hacked behavior: Circle around 3 checkpoints
hacked_rewards = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # Same time, more points!
print(f"\nHACKED (circle 3 checkpoints): {hacked_rewards}")
print(f"  Time: 10 steps, Return: {sum(hacked_rewards)}")

print("\n" + "="*60)
print("THE FIX: More careful reward design!")
print("  • Reward for completing FULL LAPS")
print("  • Penalty for revisiting same checkpoint")
print("  • Reward proportional to lap TIME (faster = better)")
print("="*60)

---
## Summary: Key Takeaways

Let's recap what we learned!

### Rewards vs Returns

| Concept | Symbol | Meaning | Analogy |
|---------|--------|---------|----------|
| **Reward** | r_t | Immediate feedback | This month's paycheck |
| **Return** | G_t | Total future reward | Lifetime savings |

### The Discounted Return Formula

```
G_t = r_t + γ·r_{t+1} + γ²·r_{t+2} + ...
    = r_t + γ·G_{t+1}  (recursive form)
```

### Discount Factor (γ)

| γ Value | Behavior | Effective Horizon |
|---------|----------|-------------------|
| 0.99 | Very patient | ~100 steps |
| 0.9 | Moderate | ~10 steps |
| 0.5 | Impatient | ~2 steps |
| 0 | Myopic | 1 step (immediate only) |

### Reward Design

- **Sparse rewards:** Simple but hard to learn
- **Dense rewards:** Easier to learn but may cause hacking
- **Agents optimize exactly what you reward!**

---
## Test Your Understanding

**1. What's the difference between reward and return?**
<details>
<summary>Click to reveal answer</summary>
Reward (r_t) is the immediate feedback at one time step. Return (G_t) is the total cumulative reward from that time step onward. Agents optimize for return, not just immediate reward.
</details>

**2. If γ = 0.9, what is the weight of a reward 3 steps in the future?**
<details>
<summary>Click to reveal answer</summary>
The weight is γ³ = 0.9³ = 0.729. So a reward of 100 in 3 steps is worth 72.9 today.
</details>

**3. Why might we prefer γ = 0.99 over γ = 0.5?**
<details>
<summary>Click to reveal answer</summary>
γ = 0.99 makes the agent care about rewards far into the future (effective horizon ~100 steps). γ = 0.5 makes the agent very short-sighted (effective horizon ~2 steps). For tasks requiring long-term planning, higher γ is better.
</details>

**4. What is reward hacking?**
<details>
<summary>Click to reveal answer</summary>
Reward hacking is when an agent finds unintended ways to maximize reward that don't align with what the designer actually wanted. Example: A robot rewarded for walking speed might just fall forward quickly.
</details>

**5. Write the recursive formula for return.**
<details>
<summary>Click to reveal answer</summary>
G_t = r_t + γ·G_{t+1}

"The return at time t equals the immediate reward plus the discounted return at the next time step."
</details>

---
## What's Next?

Excellent work! Now you understand how RL measures success through rewards and returns.

In the next notebook, we'll learn about **Policies and Value Functions** - how to represent and evaluate strategies:
- What is a policy?
- State-value functions (V)
- Action-value functions (Q)
- How value functions help us find optimal policies

**Continue to:** [Notebook 4: Policies and Value Functions](04_policies_and_value_functions.ipynb)

---

*You now understand the reward signal that drives all learning in RL!*